In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta, date

In [ ]:
###########################
# 1. Read and concatenate data
###########################
file2019_dx = "../Data/diagnosis_2019.sas7bdat"
file2020_dx = "../Data/diagnosis_2020.sas7bdat"

df2019_dx = pd.read_sas(file2019_dx)
df2020_dx = pd.read_sas(file2020_dx)
df_dx = pd.concat([df2019_dx, df2020_dx], ignore_index=True)

file2019_px = "../Data/procedures_2019.sas7bdat"
file2020_px = "../Data/procedures_2020.sas7bdat"

df2019_px = pd.read_sas(file2019_px)
df2020_px = pd.read_sas(file2020_px)
df_px = pd.concat([df2019_px, df2020_px], ignore_index=True)

In [ ]:
df_dx

In [ ]:
df_px

In [ ]:
df_dx = df_dx[["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Dx_Type" , "Dx", "PROVIDER_ID", "Enc_Type"]]
df_dx

In [ ]:
df_px = df_px[["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Px_Type" , "Px", "PROVIDER_ID", "Enc_Type"]]
df_px

In [ ]:
###########################
# 2. Decode byte columns and convert Admit_Date
###########################
for col in ['Dx_Type', 'Dx', 'PROVIDER_ID', 'Enc_Type']:
    df_dx[col] = df_dx[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

df_dx['Admit_Date'] = pd.to_datetime(df_dx['Admit_Date'])
df_dx = df_dx[df_dx['Admit_Date'].dt.year.isin([2019, 2020])]  # Ensure only 2019 and 2020 data is included

for col in ['Px_Type', 'Px', 'PROVIDER_ID', 'Enc_Type']:
    df_px[col] = df_px[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

df_px['Admit_Date'] = pd.to_datetime(df_px['Admit_Date'])
df_px = df_px[df_px['Admit_Date'].dt.year.isin([2019, 2020])]  # Ensure only 2019 and 2020 data is included

In [ ]:
df_dx

In [ ]:
df_px

In [ ]:
###########################
# 3. Classify encounter type
###########################
def classify_encounter(enc):
    if enc in ['EI', 'IP']:
        return 'Inpatient'
    elif enc == 'ED':
        return 'ED'
    elif enc in ['AV', 'IS', 'OS', 'IC', 'TH', 'OA']:
        return 'Outpatient'
    else:
        return 'Other'
        
df_dx['Encounter_Class'] = df_dx['Enc_Type'].apply(classify_encounter)
df_px['Encounter_Class'] = df_px['Enc_Type'].apply(classify_encounter)

In [ ]:
df_dx

In [ ]:
df_px

In [ ]:
###########################
# 4. Define the diagnosis code lists (without dots)
###########################

# List 1
list1_icd9 = {"9950", "99560", "99561", "99562", "99565", "99563", "99564", "99566",
              "99567", "99568", "99569", "99941", "99942", "99949"}
list1_icd10 = {"T782XXA", "T886XXA", "T7800XA", "T7801XA", "T7802XA", "T7803XA",
               "T7804XA", "T7805XA", "T7806XA", "T7807XA", "T7808XA", "T7809XA",
               "T8051XA", "T8052XA", "T8059XA"}

# List 2 (Group I)
list2_group1_icd9 = {"51911", "7861"}
list2_group1_icd10 = {"J9801", "R061"}
list2_group1_pcs = {"J1200"}
list2_group1_cpt = set()  # No CPT codes for this group

# List 2 (Group II)
list2_group2_icd9 = {"4589", "9960"}  # 99.60 is stored as "9960"
list2_group2_icd10 = {"I959", "5A12012", "5A1221Z"}
list2_group2_pcs = {"J0170", "J0171"}
list2_group2_cpt = {"92950"}

# List 3
list3_icd9 = {"9952", "99522", "99523", "99527", "99529", "9953"}
# For ICD-9, also any code in the range E930.* - E949.* will qualify.
# For ICD-10, we list the many explicit codes:
list3_icd10 = {
    "T50905A", "T410X5A", "T411X5A", "T41205A", "T41295A", "T413X5A", "T4145XA", "T8859XA",
    "T383X5A", "T50995A", "T7840XA", "T7849XA", "T360X5A", "T361X5A", "T362X5A", "T363X5A",
    "T364X5A", "T365X5A", "T366X5A", "T367X5A", "T368X5A", "T3695XA", "T370X5A", "T371X5A",
    "T372X5A", "T373X5A", "T374X5A", "T375X5A", "T378X5A", "T3795XA", "T380X5A", "T381X5A",
    "T382X5A", "T384X5A", "T385X5A", "T386X5A", "T387X5A", "T38805A", "T38815A", "T38895A",
    "T38905A", "T38995A", "T39015A", "T39095A", "T391X5A", "T392X5A", "T39315A", "T39395A",
    "T394X5A", "T398X5A", "T3995XA", "T400X5A", "T402X5A", "T403X5A", "T404X5A", "T405X5A",
    "T40605A", "T40695A", "T407X5A", "T40905A", "T40995A", "T415X5A", "T420X5A", "T421X5A",
    "T422X5A", "T423X5A", "T424X5A", "T425X5A", "T426X5A", "T4275XA", "T428X5A", "T43015A",
    "T43025A", "T431X5A", "T43205A", "T43215A", "T43225A", "T43295A", "T433X5A", "T434X5A",
    "T43505A", "T43595A", "T43605A", "T43615A", "T43625A", "T43635A", "T43695A", "T438X5A",
    "T4395XA", "T440X5A", "T441X5A", "T442X5A", "T442X5S", "T443X5A", "T444X5A", "T445X5A",
    "T446X5A", "T447X5A", "T448X5A", "T44905A", "T44905S", "T44995A", "T44995S", "T450X5A",
    "T451X5A", "T452X5A", "T453X5A", "T454X5A", "T45515A", "T45525A", "T45605A", "T45615A",
    "T45625A", "T45695A", "T457X5A", "T458X5A", "T4595XA", "T460X5A", "T461X5A", "T462X5A",
    "T463X5A", "T464X5A", "T465X5A", "T466X5A", "T467X5A", "T468X5A", "T46905A", "T46995A",
    "T470X5A", "T471X5A", "T472X5A", "T473X5A", "T474X5A", "T475X5A", "T476X5A", "T477X5A",
    "T478X5A", "T4795XA", "T480X5A", "T481X5A", "T48205A", "T48295A", "T483X5A", "T484X5A",
    "T485X5A", "T486X5A", "T48905A", "T48995A", "T490X5A", "T491X5A", "T492X5A", "T493X5A",
    "T494X5A", "T495X5A", "T496X5A", "T497X5A", "T498X5A", "T4995XA", "T500X5A", "T501X5A",
    "T502X5A", "T503X5A", "T504X5A", "T505X5A", "T506X5A", "T507X5A", "T508X5A", "T50905A",
    "T50A15A", "T50A25A", "T50A95A", "T50B15A", "T50B95A", "T50Z15A", "T50Z95A", "T887XXA"
}

### Path 1 (Wash criterion A)

In [ ]:
df_path_1_dx = df_dx[(df_dx["Encounter_Class"] == "Inpatient") | (df_dx["Encounter_Class"] == "ED")]
df_path_1_dx

In [ ]:
df_path_1_px = df_px[(df_px["Encounter_Class"] == "Inpatient") | (df_px["Encounter_Class"] == "ED")]
df_path_1_px

In [ ]:
df_path_1_dx["Dx_Type"].unique()

In [ ]:
df_path_1_px["Px_Type"].unique()

In [ ]:
df_path_1_target_icd_dx = df_path_1_dx[df_path_1_dx["Dx"].isin(list1_icd10)]
df_path_1_target_icd_dx

In [ ]:
df_path_1_target_icd_px = df_path_1_px[df_path_1_px["Px"].isin(list1_icd10)]
df_path_1_target_icd_px

In [ ]:
# First, create a copy of the dataframe to avoid modifying the original
df_grouped = df_path_1_target_icd_dx.copy()

# Create a function to calculate the max difference for each group
def calc_max_diff_days(group):
    if len(group) > 1:
        # Sort by admit date first
        group = group.sort_values('Admit_Date')
        # Calculate the day difference between consecutive dates
        day_diffs = group['Admit_Date'].diff().dt.days
        # Return the maximum difference or 0 if there are no valid differences
        return day_diffs.max()
    else:
        # For groups with only one row, return 0
        return 0

# Group by PATIENT_NUM, ENCOUNTER_NUM, Dx and apply the function
result = df_grouped.groupby(['PATIENT_NUM', 'ENCOUNTER_NUM', 'Dx']).apply(
    lambda x: pd.Series({'size': len(x), 'max_diff_days': calc_max_diff_days(x)})
).reset_index()

result

In [ ]:
result[result["size"] > 1]

In [ ]:
result_filtered = result[(result["max_diff_days"] == 0) | (result["max_diff_days"] >= 60)]
result_filtered

In [ ]:
df_path_1_final = result_filtered[["PATIENT_NUM", "ENCOUNTER_NUM"]].drop_duplicates().reset_index(drop=True)
df_path_1_final

In [ ]:
path_1_patient_num = df_path_1_final["PATIENT_NUM"].nunique()
path_1_encounter_num = df_path_1_final["ENCOUNTER_NUM"].nunique()
print(f"Number of unique patients: {path_1_patient_num}")
print(f"Number of unique encounters: {path_1_encounter_num}")

### Path 2 (Wash criterion B)

In [ ]:
df_path_2_dx = df_dx[(df_dx["Encounter_Class"] == "Outpatient")]
df_path_2_dx

In [ ]:
df_path_2_px = df_px[(df_px["Encounter_Class"] == "Outpatient")]
df_path_2_px

In [ ]:
df_path_2_dx["Dx_Type"].unique()

In [ ]:
df_path_2_px["Px_Type"].unique()

In [ ]:
df_path_2_target_icd_1_dx = df_path_2_dx[(df_path_2_dx["Dx"].isin(list1_icd10.union(list1_icd9)))]
df_path_2_target_icd_1_dx

In [ ]:
df_path_2_target_icd_1_px = df_path_2_px[(df_path_2_px["Px"].isin(list1_icd10.union(list1_icd9)))]
df_path_2_target_icd_1_px

In [ ]:
# Union list2_group1_icd10 and list2_group2_icd10
list2_icd10 = list2_group1_icd10.union(list2_group2_icd10)
list2_icd9 = list2_group1_icd9.union(list2_group2_icd9)

In [ ]:
df_path_2_target_icd_2_dx = df_path_2_dx[df_path_2_dx["Dx"].isin(list2_icd10.union(list2_icd9))]
df_path_2_target_icd_2_dx

In [ ]:
list2_pcs = list2_group1_pcs.union(list2_group2_pcs)
list2_cpt = list2_group1_cpt.union(list2_group2_cpt)

In [ ]:
df_path_2_target_icd_2_px = df_path_2_px[df_path_2_px["Px"].isin(list2_pcs.union(list2_cpt).union(list2_icd10))]
df_path_2_target_icd_2_px

In [ ]:
df_path_2_target_icd_2_px["Px_Type"].unique()

In [ ]:
df_path_2_target_icd_1_dx.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_2_target_icd_1_px.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_2_target_icd_1 = pd.concat([df_path_2_target_icd_1_dx, df_path_2_target_icd_1_px], ignore_index=True)
df_path_2_target_icd_1

In [ ]:
df_path_2_target_icd_2_dx.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_2_target_icd_2_px.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_2_target_icd_2 = pd.concat([df_path_2_target_icd_2_dx, df_path_2_target_icd_2_px], ignore_index=True)
df_path_2_target_icd_2

In [ ]:
# Join df_path_2_target_icd_1 and df_path_2_target_icd_2 on "PATIENT_NUM" and "ENCOUNTER_NUM"
df_path_2_merged = pd.merge(df_path_2_target_icd_1, df_path_2_target_icd_2, on=["PATIENT_NUM", "ENCOUNTER_NUM"], how="inner")
df_path_2_merged

In [ ]:
df_path_2_merged_filtered = df_path_2_merged[df_path_2_merged["Admit_Date_x"] == df_path_2_merged["Admit_Date_y"]]
df_path_2_merged_filtered

In [ ]:
df_path_2_final = df_path_2_merged_filtered[["PATIENT_NUM", "ENCOUNTER_NUM"]].drop_duplicates().reset_index(drop=True)
df_path_2_final

In [ ]:
path_2_patient_num = df_path_2_final["PATIENT_NUM"].nunique()
path_2_encounter_num = df_path_2_final["ENCOUNTER_NUM"].nunique()
print(f"Number of unique patients: {path_2_patient_num}")
print(f"Number of unique encounters: {path_2_encounter_num}")

### Path 3 (Wash criterion C)

In [ ]:
df_path_3_dx = df_dx[(df_dx["Encounter_Class"] == "Inpatient") | (df_dx["Encounter_Class"] == "ED")]
df_path_3_dx

In [ ]:
df_path_3_dx["Dx_Type"].unique()

In [ ]:
df_path_3_px = df_px[(df_px["Encounter_Class"] == "Inpatient") | (df_px["Encounter_Class"] == "ED")]
df_path_3_px

In [ ]:
df_path_3_px["Px_Type"].unique()

In [ ]:
df_path_3_target_icd_1_dx = df_path_3_dx[df_path_3_dx["Dx"].isin(list3_icd10)]
df_path_3_target_icd_1_dx

In [ ]:
df_path_3_target_icd_1_px = df_path_3_px[df_path_3_px["Px"].isin(list3_icd10)]
df_path_3_target_icd_1_px

In [ ]:
df_path_3_target_icd_2_dx = df_path_3_dx[df_path_3_dx["Dx"].isin(list2_group1_icd10)]
df_path_3_target_icd_2_dx

In [ ]:
df_path_3_target_icd_2_px = df_path_3_px[df_path_3_px["Px"].isin(list2_group1_icd10.union(list2_group1_pcs).union(list2_group1_cpt))]
df_path_3_target_icd_2_px

In [ ]:
df_path_3_target_icd_3_dx = df_path_3_dx[df_path_3_dx["Dx"].isin(list2_group2_icd10)]
df_path_3_target_icd_3_dx

In [ ]:
df_path_3_target_icd_3_px = df_path_3_px[df_path_3_px["Px"].isin(list2_group2_icd10.union(list2_group2_pcs).union(list2_group2_cpt))]
df_path_3_target_icd_3_px

In [ ]:
df_path_3_target_icd_1_dx.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_3_target_icd_1_px.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_3_target_icd_1 = pd.concat([df_path_3_target_icd_1_dx, df_path_3_target_icd_1_px], ignore_index=True)
df_path_3_target_icd_1

In [ ]:
df_path_3_target_icd_2_dx.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_3_target_icd_2_px.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_3_target_icd_2 = pd.concat([df_path_3_target_icd_2_dx, df_path_3_target_icd_2_px], ignore_index=True)
df_path_3_target_icd_2

In [ ]:
df_path_3_target_icd_3_dx.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_3_target_icd_3_px.columns = ["PATIENT_NUM", "ENCOUNTER_NUM", "Admit_Date", "Type", "Code", "PROVIDER_ID", "Enc_Type", "Encounter_Class"]
df_path_3_target_icd_3 = pd.concat([df_path_3_target_icd_3_dx, df_path_3_target_icd_3_px], ignore_index=True)
df_path_3_target_icd_3

In [ ]:
# Join df_path_3_target_icd_1, df_path_3_target_icd_2, and df_path_3_target_icd_3 on "PATIENT_NUM" and "ENCOUNTER_NUM"
df_path_3_merged_1 = pd.merge(df_path_3_target_icd_1, df_path_3_target_icd_2, on=["PATIENT_NUM", "ENCOUNTER_NUM"], how="inner")
df_path_3_merged_1

In [ ]:
df_path_3_merged = pd.merge(df_path_3_merged_1, df_path_3_target_icd_3, on=["PATIENT_NUM", "ENCOUNTER_NUM"], how="inner")
df_path_3_merged

In [ ]:
df_path_3_merged_filtered = df_path_3_merged[(df_path_3_merged["Admit_Date_x"] == df_path_3_merged["Admit_Date_y"]) & (df_path_3_merged["Admit_Date_x"] == df_path_3_merged["Admit_Date"])]
df_path_3_merged_filtered

In [ ]:
df_path_3_final = df_path_3_merged_filtered[["PATIENT_NUM", "ENCOUNTER_NUM"]].drop_duplicates().reset_index(drop=True)
df_path_3_final

In [ ]:
path_3_patient_num = df_path_3_final["PATIENT_NUM"].nunique()
path_3_encounter_num = df_path_3_final["ENCOUNTER_NUM"].nunique()
print(f"Number of unique patients: {path_3_patient_num}")
print(f"Number of unique encounters: {path_3_encounter_num}")

### All Paths

In [ ]:
path_all_final = pd.concat([df_path_1_final, df_path_2_final, df_path_3_final], ignore_index=True)
path_all_final

In [ ]:
path_all_final = path_all_final[['PATIENT_NUM', 'ENCOUNTER_NUM']].drop_duplicates().reset_index(drop=True)
path_all_final

In [ ]:
path_all_patient_num = path_all_final["PATIENT_NUM"].nunique()
path_all_encounter_num = path_all_final["ENCOUNTER_NUM"].nunique()
print(f"Number of unique patients: {path_all_patient_num}")
print(f"Number of unique encounters: {path_all_encounter_num}")

In [ ]:
path_all_final.to_csv("path_all_final.csv", index=False)

### New: prepare input for cTAKES

In [ ]:
df_target_encounter = pd.read_csv("path_all_final.csv")
df_target_encounter

In [ ]:
df_all_notes = pd.read_csv("/PHShome/bg615/Anaphylaxis_pipeline/Data/FE5_notes_all.csv", usecols=["PATIENT_NUM", "ENCOUNTER_NUM", "UPDATE_DATE", "PROVIDER_ID", "OBSERVATION_BLOB"])
df_all_notes

In [ ]:
df_all_notes.rename(columns={"ENCOUNTER_NUM": "TEXT_ID"}, inplace=True)
df_all_notes

In [ ]:
df_text_id_to_encounter_id = pd.read_csv("/PHShome/bg615/Anaphylaxis_pipeline/Data/text_id_to_encounter_id.csv")
df_text_id_to_encounter_id

In [ ]:
df_all_notes_merged = pd.merge(df_all_notes, df_text_id_to_encounter_id, on=["TEXT_ID"], how="inner")
df_all_notes_merged

In [ ]:
df_all_notes_merged.rename(columns={"EncounterID_Final": "ENCOUNTER_NUM"}, inplace=True)
df_all_notes_merged

In [ ]:
df_merged_new = pd.merge(df_all_notes_merged, df_target_encounter, on=["PATIENT_NUM"], how="inner")
df_merged_new

In [ ]:
df_merged_new.rename(columns={"ENCOUNTER_NUM_x": "ENCOUNTER_NUM"}, inplace=True)
df_merged_new.drop(columns=["ENCOUNTER_NUM_y"], inplace=True)
df_merged_new

In [ ]:
df_merged_new.to_csv("/PHShome/bg615/Anaphylaxis_pipeline/Data/EHR Notes/anaphylaxis_notes.csv", index=False)

In [ ]:
df_merged_new[['PATIENT_NUM', 'ENCOUNTER_NUM']].drop_duplicates().reset_index(drop=True)